# Test OpenAI Agents SDK
- Implement a workflow to write a daily AI newsletter

In [1]:
import os
import yaml
import dotenv
import logging
import json
import yaml
from datetime import datetime
import time
import random
import glob
import pickle
import sqlite3

from pathlib import Path

import asyncio
import nest_asyncio

import pydantic
from pydantic import BaseModel, Field, RootModel
from typing import Dict, TypedDict, Type, List, Optional, Any, Iterable
from dataclasses import dataclass, field
from enum import Enum

import numpy as np
import pandas as pd

import pandas as pd
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import hdbscan

import openai
from openai import AsyncOpenAI

import agents
from agents.exceptions import InputGuardrailTripwireTriggered
from agents import (Agent, Runner, Tool, OpenAIResponsesModel, 
                    ModelSettings, FunctionTool, InputGuardrail, GuardrailFunctionOutput,
                    SQLiteSession, set_default_openai_api, set_default_openai_client
                   )


import tenacity
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

from IPython.display import HTML, Image, Markdown, display

from log_handler import SQLiteLogHandler, setup_sqlite_logging, sanitize_error_for_logging
from config import LOGDB
from llm import LLMagent, LangfuseClient  # methods to apply prompts async to large batches
from db import Url 

from fetch import Fetcher # fetch news urls
from newsletter_state import NewsletterAgentState, StepStatus
from news_agent import NewsletterAgent


In [2]:
print(f"OpenAI:            {openai.__version__}")
print(f"OpenAI Agents SDK  {agents.__version__}")
print(f"Pydantic           {pydantic.__version__}")


OpenAI:            1.109.0
OpenAI Agents SDK  0.3.1
Pydantic           2.11.9


In [3]:
dotenv.load_dotenv()

# to run async in jupyter notebook
nest_asyncio.apply()

# verbose OpenAI console logging if something doesn't work
# logging.basicConfig(level=logging.DEBUG)
# openai_logger = logging.getLogger("openai")
# openai_logger.setLevel(logging.DEBUG)


In [4]:
# modules create a default logger, or we can pass this logger

def setup_logging(session_id: str = "default", db_path: str = "agent_logs.db") -> logging.Logger:
    """Set up logging to console and SQLite database."""

    # Create logger
    logging.basicConfig(level=logging.INFO)

    logger = logging.getLogger(f"NewsletterAgent.{session_id}")
    logger.setLevel(logging.INFO)

    # Clear any existing handlers
    logger.handlers.clear()

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_formatter = logging.Formatter(
        '%(asctime)s | %(name)s | %(levelname)s | %(message)s',
        datefmt='%H:%M:%S'
    )
    console_handler.setFormatter(console_formatter)

    # SQLite handler
    sqlite_handler = SQLiteLogHandler(db_path)
    sqlite_handler.setLevel(logging.INFO)
    sqlite_formatter = logging.Formatter('%(message)s')
    sqlite_handler.setFormatter(sqlite_formatter)

    # Add handlers to logger
    logger.addHandler(console_handler)
    logger.addHandler(sqlite_handler)

    # Prevent propagation to root logger
    logger.propagate = False

    return logger

logger = setup_logging("newsletter_agent", "test_logs.db")

# Log some test messages
logger.info("Test info message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.warning("Test warning message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.error("Test error message", extra={
    'step_name': 'error_step',
    'agent_session': 'demo_session'
})

sanitize_error_for_logging("log with some bad stuff for the filter: sk-proj-123456789012345678901234567890123456789012345678")

11:52:57 | NewsletterAgent.newsletter_agent | INFO | Test info message
11:52:57 | NewsletterAgent.newsletter_agent | WARNING | Test warning message
11:52:57 | NewsletterAgent.newsletter_agent | ERROR | Test error message


'log with some bad stuff for the filter: [API_KEY_REDACTED]'

# Run Agent Worfklow

In [5]:
print("🚀 Creating NewsletterAgent...")

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

# Set up OpenAI client for the agents SDK
set_default_openai_client(AsyncOpenAI(api_key=api_key))

# set up state
# session_id = 'test_newsletter_20250923174350688839'
# step_name = 'step_05_cluster_by_topic'
# del session_id

do_download=False
process_since='2025-09-24 18:00:00'

# Create agent with persistent state
if 'session_id' in vars():
    # load state from db for session_id and state
    print("session_id is defined")
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db", 
                                 do_download=do_download,
                                 process_since=process_since)
    state = state.load_from_db(step_name)
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=True, timeout=30)    
else:
    # create new session
    print("session_id is not defined")
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")    
    session_id = f"test_newsletter_{timestamp}"
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db",
                                 do_download=do_download,
                                 process_since=process_since
                                ) 
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=False, timeout=30)
    state.serialize_to_db("initialize")

🚀 Creating NewsletterAgent...
session_id is not defined
test_newsletter_20250925115257412043


In [6]:
state.get_status()

{'headlines': {'total': 0},
 'sources': {'config_file': 'sources.yaml', 'loaded_sources': 0},
 'topics': {'cluster_topics': 0, 'topics': []},
 'workflow': {'current_step': 'step_01_fetch_urls',
  'workflow_complete': False,
  'workflow_status': 'not_started',
  'workflow_status_message': '',
  'progress_percentage': 0.0,
  'max_edits': 2,
  'concurrency': 16},
 'processing': {'topic_clusters': 0,
  'newsletter_sections': 0,
  'final_newsletter_length': 0}}

In [7]:
state.get_current_step()


'step_01_fetch_urls'

In [8]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


📝 User prompt: 'Show the workflow status'


11:53:06 | NewsletterAgent.test_newsletter_20250925115257412043 | INFO | Starting check_workflow_status
11:53:06 | NewsletterAgent.test_newsletter_20250925115257412043 | INFO | Completed check_workflow_status


⏱️  Total execution time: 6.57s
📊 Final result:
Current workflow status:

- Progress: 0.0% (0/9 complete)
- Status summary: 0 complete, 0 started, 0 failed, 9 not started
- Next step: Step 1 — Fetch URLs

Step details:
- Step 1: Fetch Urls — not_started
- Step 2: Filter Urls — not_started
- Step 3: Download Articles — not_started
- Step 4: Extract Summaries — not_started
- Step 5: Cluster By Topic — not_started
- Step 6: Rate Articles — not_started
- Step 7: Select Sections — not_started
- Step 8: Draft Sections — not_started
- Step 9: Finalize Newsletter — not_started

What would you like me to do next? (Run all steps, start from a specific step, or resume/continue.)


In [9]:
# User prompt to run a workflow step
user_prompt = "Run step 1, fetch urls"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)



📝 User prompt: 'Run step 1, fetch urls'


11:53:14 | NewsletterAgent.test_newsletter_20250925115257412043 | INFO | Starting check_workflow_status
11:53:14 | NewsletterAgent.test_newsletter_20250925115257412043 | INFO | Completed check_workflow_status
11:53:16 | NewsletterAgent.test_newsletter_20250925115257412043 | INFO | Starting Step 1: Gather URLs
2025-09-25 11:53:16,009 - fetcher_5437863184 - INFO - [fetcher_init] Loading sources from sources.yaml
2025-09-25 11:53:16,022 - fetcher_5437863184 - INFO - [fetcher_init] Loaded 17 sources: 7 RSS, 9 HTML, 1 API
2025-09-25 11:53:16,024 - fetcher_5437863184 - DEBUG - [fetcher_sources] Source 'Ars Technica': type=RSS, url=https://arstechnica.com/ai/
2025-09-25 11:53:16,024 - fetcher_5437863184 - DEBUG - [fetcher_sources] Source 'Bloomberg': type=RSS, url=https://www.bloomberg.com/ai
2025-09-25 11:53:16,025 - fetcher_5437863184 - DEBUG - [fetcher_sources] Source 'Business Insider': type=html, url=https://www.businessinsider.com/tech
2025-09-25 11:53:16,025 - fetcher_5437863184 - DEBU

2025-09-25 11:53:16,333 - fetcher_5437863184 - INFO - [fetch_html] Parsing HTML file: download/sources/VentureBeat.html
2025-09-25 11:53:16,340 - fetcher_5437863184 - INFO - [fetch_html] Parsed HTML file: download/sources/VentureBeat.html
2025-09-25 11:53:16,340 - fetcher_5437863184 - INFO - [fetch_html] HTML fetch successful for VentureBeat: 12 articles
2025-09-25 11:53:16,341 - fetcher_5437863184 - INFO - [fetch_html] Using existing HTML file from WSJ: https://www.wsj.com/tech/ai
2025-09-25 11:53:16,341 - fetcher_5437863184 - INFO - [fetch_html] Parsing HTML file: download/sources/WSJ.html
2025-09-25 11:53:16,363 - fetcher_5437863184 - INFO - [fetch_html] Parsed HTML file: download/sources/WSJ.html
2025-09-25 11:53:16,363 - fetcher_5437863184 - INFO - [fetch_html] HTML fetch successful for WSJ: 26 articles
2025-09-25 11:53:16,363 - fetcher_5437863184 - INFO - [fetch_html] Using existing HTML file from Washington Post: https://www.washingtonpost.com/technology/innovations/
2025-09-25 

,source,url
0,Ars Technica,20
1,Bloomberg,25
2,Business Insider,17
3,FT,48
4,Feedly AI,73
5,Hacker News,30
6,HackerNoon,50
7,New York Times,26
8,NewsAPI,94
9,Reddit,53


,source,title,url,published,rss_summary,id
0,Ars Technica,DeepMind’s robotic ballet: An AI for coordinat...,https://arstechnica.com/science/2025/09/deepmi...,"Thu, 25 Sep 2025 11:15:40 +0000",An AI figures out how robots can get jobs done...,0
1,Ars Technica,Why does OpenAI need six giant data centers?,https://arstechnica.com/ai/2025/09/why-does-op...,"Wed, 24 Sep 2025 16:06:03 +0000",OpenAI's new $400 billion announcement reveals...,1
2,Ars Technica,When “no” means “yes”: Why AI chatbots can’t p...,https://arstechnica.com/ai/2025/09/when-no-mea...,"Tue, 23 Sep 2025 22:23:22 +0000",New study examines how a helpful AI response c...,2
3,Ars Technica,OpenAI and Nvidia’s $100B AI plan will require...,https://arstechnica.com/ai/2025/09/openai-and-...,"Mon, 22 Sep 2025 19:17:28 +0000","""This is a giant project,"" Nvidia CEO said of ...",3
4,Ars Technica,DeepMind AI safety report explores the perils ...,https://arstechnica.com/google/2025/09/deepmin...,"Mon, 22 Sep 2025 18:18:00 +0000",DeepMind releases version 3.0 of its AI Fronti...,4
...,...,...,...,...,...,...
608,NewsAPI,Step into the future: The full AI Stage at Tec...,https://biztoc.com/x/bca313645c5d345d,2025-09-24T14:41:00Z,NaN,608
609,NewsAPI,Bitcoin Miners Surge on Speculation of OpenAI-...,https://biztoc.com/x/a7b611dec662f738,2025-09-24T15:48:07Z,NaN,609
610,NewsAPI,Startup using AI to automate software testing ...,https://biztoc.com/x/856c60081122fcf0,2025-09-24T13:12:10Z,NaN,610
611,NewsAPI,The importance of scientific research in build...,https://biztoc.com/x/e66df74bc285ad44,2025-09-24T13:12:13Z,NaN,611


11:53:17 | NewsletterAgent.test_newsletter_20250925115257412043 | INFO | Completed Step 1: Gathered 693 articles
11:53:19 | NewsletterAgent.test_newsletter_20250925115257412043 | INFO | Starting check_workflow_status
11:53:19 | NewsletterAgent.test_newsletter_20250925115257412043 | INFO | Completed check_workflow_status


⏱️  Total execution time: 11.57s
📊 Final result:
Step 1 (Fetch URLs) completed.

Summary:
- Collected 693 article records from 17 sources via RSS; 613 articles stored in persistent state.
- Workflow progress: 1/9 steps complete (11.1%).
- Next step: Step 2 — Filter URLs (to AI-related content).

Would you like me to proceed to Step 2 now?


In [10]:
pd.DataFrame(state.headline_data) 


,source,title,url,published,rss_summary,id
0,Ars Technica,DeepMind’s robotic ballet: An AI for coordinat...,https://arstechnica.com/science/2025/09/deepmi...,"Thu, 25 Sep 2025 11:15:40 +0000",An AI figures out how robots can get jobs done...,0
1,Ars Technica,Why does OpenAI need six giant data centers?,https://arstechnica.com/ai/2025/09/why-does-op...,"Wed, 24 Sep 2025 16:06:03 +0000",OpenAI's new $400 billion announcement reveals...,1
2,Ars Technica,When “no” means “yes”: Why AI chatbots can’t p...,https://arstechnica.com/ai/2025/09/when-no-mea...,"Tue, 23 Sep 2025 22:23:22 +0000",New study examines how a helpful AI response c...,2
3,Ars Technica,OpenAI and Nvidia’s $100B AI plan will require...,https://arstechnica.com/ai/2025/09/openai-and-...,"Mon, 22 Sep 2025 19:17:28 +0000","""This is a giant project,"" Nvidia CEO said of ...",3
4,Ars Technica,DeepMind AI safety report explores the perils ...,https://arstechnica.com/google/2025/09/deepmin...,"Mon, 22 Sep 2025 18:18:00 +0000",DeepMind releases version 3.0 of its AI Fronti...,4
...,...,...,...,...,...,...
608,NewsAPI,Step into the future: The full AI Stage at Tec...,https://biztoc.com/x/bca313645c5d345d,2025-09-24T14:41:00Z,NaN,608
609,NewsAPI,Bitcoin Miners Surge on Speculation of OpenAI-...,https://biztoc.com/x/a7b611dec662f738,2025-09-24T15:48:07Z,NaN,609
610,NewsAPI,Startup using AI to automate software testing ...,https://biztoc.com/x/856c60081122fcf0,2025-09-24T13:12:10Z,NaN,610
611,NewsAPI,The importance of scientific research in build...,https://biztoc.com/x/e66df74bc285ad44,2025-09-24T13:12:13Z,NaN,611


In [11]:
countdf = pd.DataFrame(state.headline_data) \
    .groupby("source") \
    .count()[["id"]] \
    .reset_index() \
    .rename(columns={'id': 'count'}) \
    .sort_values("count", ascending=False)
countdf 


,source,count
8,NewsAPI,94
4,Feedly AI,73
9,Reddit,53
6,HackerNoon,50
12,The Register,50
3,FT,48
13,The Verge,30
5,Hacker News,30
16,Washington Post,28
7,New York Times,26


In [12]:
# Run tool directly without LLM processing an input prompt or results
# user_prompt = "Run step 2, filter urls"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("filter_urls")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


11:53:21 | NewsletterAgent.test_newsletter_20250925115257412043 | INFO | Starting Step 2: Filter URLs
INFO:llm:Initialized LangfuseClient


<class 'datetime.datetime'>
2025-09-25 09:49:26.733280
<class 'datetime.datetime'>
2025-09-25 09:49:26.733394
<class 'datetime.datetime'>
2025-09-25 09:49:26.733460
<class 'datetime.datetime'>
2025-09-25 09:49:26.733526
<class 'datetime.datetime'>
2025-09-25 09:49:26.733580
<class 'datetime.datetime'>
2025-09-25 09:49:26.733650
<class 'datetime.datetime'>
2025-09-25 09:49:26.733709
<class 'datetime.datetime'>
2025-09-25 09:49:26.733826
<class 'datetime.datetime'>
2025-09-25 09:49:26.733889
<class 'datetime.datetime'>
2025-09-25 09:49:26.733956
<class 'datetime.datetime'>
2025-09-25 09:49:26.734015
<class 'datetime.datetime'>
2025-09-25 09:49:26.734079
<class 'datetime.datetime'>
2025-09-25 09:49:26.734140
<class 'datetime.datetime'>
2025-09-25 09:49:26.734204
<class 'datetime.datetime'>
2025-09-25 09:49:26.734261
<class 'datetime.datetime'>
2025-09-25 09:49:26.734360
<class 'datetime.datetime'>
2025-09-25 09:49:26.734423
<class 'datetime.datetime'>
2025-09-25 09:49:26.734501
<class 'da

INFO:llm:Successfully retrieved prompt 'newsagent/filter_urls' from Langfuse
INFO:llm:Parsed prompt 'newsagent/filter_urls': model=gpt-4.1-mini, system_len=458, user_len=954
11:53:57 | NewsletterAgent.test_newsletter_20250925115257412043 | INFO | Completed Step 2: 336 AI-related articles


⏱️  Total execution time: 35.41s
📊 Final result:
✅ Step 2 completed successfully! Filtered 613 headlines to 336 AI-related articles.

📊 Results stored in persistent state. Current step: step_03_download_articles


In [ ]:
# User prompt to run workflow
# user_prompt = "Run step 3, download full articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("download_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

11:54:43 | NewsletterAgent.test_newsletter_20250925115257412043 | INFO | Starting Step 3: Download Articles
11:54:43 | NewsletterAgent.test_newsletter_20250925115257412043 | INFO | Starting concurrent scraping of 336 AI-related articles
11:54:43 | NewsletterAgent.test_newsletter_20250925115257412043 | INFO | Launching browser for 336 URLs with 16 concurrent workers
11:54:45 | NewsletterAgent.test_newsletter_20250925115257412043 | INFO | Worker 0 fetching 1 of 336 https://go.theregister.com/feed/www.theregister.com/2025/09/23/kaspersky_revengehotels_checks_back_in/
11:54:45 | NewsletterAgent.test_newsletter_20250925115257412043 | INFO | scrape_url(https://go.theregister.com/feed/www.theregister.com/2025/09/23/kaspersky_revengehotels_checks_back_in/)
11:54:45 | NewsletterAgent.test_newsletter_20250925115257412043 | INFO | scraping https://go.theregister.com/feed/www.theregister.com/2025/09/23/kaspersky_revengehotels_checks_back_in/ to download/html
11:54:45 | NewsletterAgent.test_newslet

11:54:46 | NewsletterAgent.test_newsletter_20250925115257412043 | INFO | scraping https://arstechnica.com/health/2025/09/ai-medical-tools-found-to-downplay-symptoms-of-women-ethnic-minorities/ to download/html
11:54:46 | NewsletterAgent.test_newsletter_20250925115257412043 | INFO | Downloading https://arstechnica.com/health/2025/09/ai-medical-tools-found-to-downplay-symptoms-of-women-ethnic-minorities/
11:54:46 | NewsletterAgent.test_newsletter_20250925115257412043 | INFO | Worker 11 fetching 11 of 336 https://biztoc.com/x/c31a2cf8b2a32750
11:54:46 | NewsletterAgent.test_newsletter_20250925115257412043 | INFO | scrape_url(https://biztoc.com/x/c31a2cf8b2a32750)
11:54:46 | NewsletterAgent.test_newsletter_20250925115257412043 | INFO | scraping https://biztoc.com/x/c31a2cf8b2a32750 to download/html
11:54:46 | NewsletterAgent.test_newsletter_20250925115257412043 | INFO | Downloading https://biztoc.com/x/c31a2cf8b2a32750
11:54:46 | NewsletterAgent.test_newsletter_20250925115257412043 | INFO 

In [ ]:
#     initial_url: str
#     final_url: str
#     title: str
#     isAI: bool
#     created_at: Optional[datetime]
headline_df = agent.state.headline_dict.copy()

In [ ]:
headline_df

In [ ]:
import sqlite3
import db

with sqlite3.connect("newsletter_agent.db") as conn:
    db.Url.create_table(conn)
    for row in headline_df.itertuples():
        print(row.url, '', row.title, row.isAI, datetime.now())
        myurl = db.Url(row.url, '', row.title, row.isAI, datetime.now())
        myurl.insert(conn)


In [ ]:
# User prompt to run workflow
# user_prompt = "Run step 4, Summarize articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("extract_summaries")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

In [ ]:
# User prompt to run workflow
# user_prompt = "Run step 5, Cluster articles by topic"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("cluster_by_topic")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


In [ ]:
state.headline_dict.loc[state.headline_dict["url"] != state.headline_dict["final_url"]]


In [ ]:
state.headline_dict.columns

In [ ]:

    def create_extended_summary(row):
        parts = []

        # Add title if present
        if 'title' in row and row['title']:
            parts.append(str(row['title']).strip())

        # Add description if present
        if 'description' in row and row['description']:
            parts.append(str(row['description']).strip())

        # Add topics if present (join with commas)
        if 'topics' in row and row['topics']:
            if isinstance(row['topics'], list):
                topics_str = ", ".join(str(topic).strip() for topic in row['topics'] if topic)
            else:
                topics_str = str(row['topics']).strip()
            if topics_str:
                parts.append(topics_str)

        # Add summary if present
        if pd.notna(row.get('summary')) and row.get('summary'):
            parts.append(str(row['summary']).strip())

        return "\n\n".join(parts)

    async def _get_embeddings_df(self, headline_data: pd.DataFrame, embedding_model: str = "text-embedding-3-large") -> pd.DataFrame:
        """
        Get embeddings for article summaries and return as DataFrame.

        Args:
            headline_data: DataFrame with articles containing summary column
            embedding_model: OpenAI embedding model to use

        Returns:
            DataFrame with embeddings for each extended summary
        """
        from openai import OpenAI
        from llm import paginate_df_async

        # Create extended_summary column by concatenating available fields
        headline_data_copy = headline_data.copy()

        headline_data_copy['extended_summary'] = headline_data_copy.apply(create_extended_summary, axis=1)

        # Filter to articles with non-empty extended summaries
        articles_with_summaries = headline_data_copy[
            (headline_data_copy['extended_summary'].notna()) &
            (headline_data_copy['extended_summary'] != '')
        ].copy()

        all_embeddings = []
        client = OpenAI()

        # Use paginate_df_async similar to dedupe_by_cosine_similarity.py
        async for batch_df in paginate_df_async(articles_with_summaries, 25):
            text_batch = batch_df["extended_summary"].to_list()
            response = client.embeddings.create(input=text_batch, model=embedding_model)
            batch_embeddings = [item.embedding for item in response.data]
            all_embeddings.extend(batch_embeddings)

        # Create DataFrame with embeddings, preserving original index
        embedding_df = pd.DataFrame(
            all_embeddings,
            index=articles_with_summaries.index
        )

        return embedding_df


In [ ]:
headline_df = state.headline_dict
headline_df['extended_summary'] = headline_df.apply(create_extended_summary, axis=1)


embeddings_df = await _get_embeddings_df(_, state.headline_dict)

In [ ]:
embeddings_df

In [ ]:
n_components = 60
min_cluster_size = 4
min_samples =3 



In [ ]:
from sklearn.decomposition import TruncatedSVD
RANDOM_STATE = 42

svd = TruncatedSVD(n_components=n_components, random_state=RANDOM_STATE)
reduced_embeddings = svd.fit_transform(embeddings_df)
# Re-normalize after SVD
reduced_embeddings /= np.linalg.norm(reduced_embeddings, axis=1, keepdims=True)


In [ ]:
# Fit HDBSCAN
print("=== HDBSCAN Parameters ===")
print(f"min_cluster_size:   {min_cluster_size}")
print(f"min_samples:        {min_samples}")
print(f"n_components:       {n_components}")
clusterer = hdbscan.HDBSCAN(
    min_cluster_size=min_cluster_size,
    min_samples=min_samples,
    metric="euclidean",
    cluster_selection_method="eom",
)

labels = clusterer.fit_predict(reduced_embeddings)



In [ ]:
def calculate_clustering_metrics(embeddings_array, labels, clusterer=None):
    """
    Calculate various clustering quality metrics for HDBSCAN results.
    
    Args:
        embeddings_array: Original normalized embeddings used for clustering
        labels: Cluster labels from HDBSCAN
        clusterer: Optional HDBSCAN clusterer object
    
    Returns:
        Dictionary of clustering metrics
    """
    
    # Filter out noise points (-1 labels) for some metrics
    non_noise_mask = labels != -1
    non_noise_embeddings = embeddings_array[non_noise_mask]
    non_noise_labels = labels[non_noise_mask]
    
    metrics = {}
    
    # Basic cluster statistics
    unique_labels = set(labels)
    n_clusters = len(unique_labels) - (1 if -1 in unique_labels else 0)
    n_noise = np.sum(labels == -1)
    
    metrics['n_clusters'] = n_clusters
    metrics['n_noise_points'] = n_noise
    metrics['noise_ratio'] = n_noise / len(labels)
    
    # Cluster size distribution
    cluster_sizes = Counter(labels[labels != -1])
    if cluster_sizes:
        metrics['avg_cluster_size'] = np.mean(list(cluster_sizes.values()))
        metrics['std_cluster_size'] = np.std(list(cluster_sizes.values()))
        metrics['min_cluster_size'] = min(cluster_sizes.values())
        metrics['max_cluster_size'] = max(cluster_sizes.values())
    
    # Skip other metrics if we have too few clusters or too much noise
    if n_clusters < 2 or len(non_noise_labels) < 2:
        print("Warning: Too few clusters or too much noise for some metrics")
        return metrics
    
    # HDBSCAN-specific metrics
    # gives some divide by 0 errors
    if clusterer is not None:
        try:
            # Validity index (HDBSCAN's internal metric)
            validity_idx = hdbscan.validity.validity_index(
                embeddings_array, labels, metric='euclidean'
            )
            metrics['hdbscan_validity_index'] = validity_idx
        except Exception as e:
            print(f"Could not compute HDBSCAN validity index: {e}")
        
        # Cluster persistence (stability)
        if hasattr(clusterer, 'cluster_persistence_'):
            metrics['cluster_persistence'] = clusterer.cluster_persistence_
    
    # Scikit-learn clustering metrics (excluding noise points)
    try:
        # Silhouette Score (higher is better, range [-1, 1])
        sil_score = silhouette_score(non_noise_embeddings, non_noise_labels, metric='euclidean')
        metrics['silhouette_score'] = sil_score
        
        # Calinski-Harabasz Index (higher is better)
        ch_score = calinski_harabasz_score(non_noise_embeddings, non_noise_labels)
        metrics['calinski_harabasz_score'] = ch_score
        
        # Davies-Bouldin Index (lower is better)
        db_score = davies_bouldin_score(non_noise_embeddings, non_noise_labels)
        metrics['davies_bouldin_score'] = db_score
        
    except Exception as e:
        print(f"Could not compute sklearn metrics: {e}")
    
    # Custom composite score balancing cluster quality and quantity
    if 'silhouette_score' in metrics and n_clusters > 0:
        # Penalize too many small clusters or too few large clusters
        cluster_balance = 1 / (1 + abs(np.log(n_clusters / 10)))  # Optimal around 10 clusters
        size_consistency = 1 / (1 + metrics.get('std_cluster_size', 0) / max(metrics.get('avg_cluster_size', 1), 1))
        noise_penalty = 1 - min(metrics['noise_ratio'], 0.5)  # Penalize high noise
        
        composite_score = (
            0.5 * max(metrics['silhouette_score'], 0) +  # Quality component
            0.5 * max(metrics['hdbscan_validity_index'], 0)
#             0.1 * cluster_balance +                       # Quantity component  
#             0.1 * size_consistency +                      # Size consistency
#             0.3 * noise_penalty                           # Noise penalty
        )
        metrics['composite_score'] = composite_score
    
    return metrics

def print_clustering_summary(metrics):
    """Print a nice summary of clustering metrics."""
    print("=== Clustering Quality Metrics ===")
    print(f"Number of clusters: {metrics.get('n_clusters', 'N/A')}")
    print(f"Noise points: {metrics.get('n_noise_points', 'N/A')} ({metrics.get('noise_ratio', 0):.1%})")
    
    if 'avg_cluster_size' in metrics:
        print(f"Average cluster size: {metrics['avg_cluster_size']:.1f} ± {metrics.get('std_cluster_size', 0):.1f}")
        print(f"Cluster size range: {metrics.get('min_cluster_size', 'N/A')} - {metrics.get('max_cluster_size', 'N/A')}")
    
    print("=== Quality Scores ===")
    if 'silhouette_score' in metrics:
        print(f"Silhouette Score: {metrics['silhouette_score']:.3f} (higher is better)")
    if 'calinski_harabasz_score' in metrics:
        print(f"Calinski-Harabasz Score: {metrics['calinski_harabasz_score']:.1f} (higher is better)")
    if 'davies_bouldin_score' in metrics:
        print(f"Davies-Bouldin Score: {metrics['davies_bouldin_score']:.3f} (lower is better)")
    if 'hdbscan_validity_index' in metrics:
        print(f"HDBSCAN Validity Index: {metrics['hdbscan_validity_index']:.3f}")
    if 'composite_score' in metrics:
        print(f"Composite Score: {metrics['composite_score']:.3f} (higher is better)")
    print()



In [ ]:
from collections import Counter
import optuna

# Calculate metrics
metrics = calculate_clustering_metrics(reduced_embeddings, labels, clusterer)
print_clustering_summary(metrics)


In [ ]:
MIN_COMPONENTS = 20
def objective(trial, embeddings_array):

    n_components = trial.suggest_int('n_components', 
                                     MIN_COMPONENTS, 
                                     embeddings_array.shape[1] // 4)  
    
    svd = TruncatedSVD(n_components=n_components, random_state=RANDOM_STATE)
    reduced_embeddings = svd.fit_transform(embeddings_array)
    # Re-normalize after SVD
    reduced_embeddings /= np.linalg.norm(reduced_embeddings, axis=1, keepdims=True)

    # HDBSCAN hyperparameters to optimize
    min_cluster_size = trial.suggest_int('min_cluster_size', 2, 10)
    min_samples = trial.suggest_int('min_samples', 2, min_cluster_size)

    # Fit HDBSCAN
    print("=== HDBSCAN Parameters ===")
    print(f"min_cluster_size:   {min_cluster_size}")
    print(f"min_samples:        {min_samples}")
    print(f"n_components:       {n_components}")
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size,
        min_samples=min_samples,
        metric="euclidean",
        cluster_selection_method="eom",
    )

    labels = clusterer.fit_predict(reduced_embeddings)

    # Calculate metrics
    metrics = calculate_clustering_metrics(reduced_embeddings, labels, clusterer)
    print_clustering_summary(metrics)

    # Return negative composite score (Optuna minimizes)
    composite_score = metrics.get('composite_score', -1.0)

    # Penalize if no valid clusters found or too much noise
    if metrics.get('n_clusters', 0) < 2 or metrics.get('noise_ratio', 1.0) > 0.8:
        composite_score = -1.0

    return -composite_score    



In [ ]:
def optimize_hdbscan(embeddings_array, n_trials=100, timeout=None):
    """
    Optimize HDBSCAN hyperparameters using Optuna.
    
    Args:
        embeddings_array: Normalized embeddings array
        n_trials: Number of optimization trials
        timeout: Maximum time in seconds (None for no limit)
    
    Returns:
        Dictionary with best parameters and results
    """
    
    print(f"Starting optimization with {n_trials} trials...")
    print(f"Original embedding shape: {embeddings_array.shape}")
    
    # Create study
    study = optuna.create_study(
        direction='minimize',  # We return negative composite score
        sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE),
        pruner=optuna.pruners.MedianPruner(n_startup_trials=10)
    )
    
    # Optimize
    study.optimize(
        lambda trial: objective(trial, embeddings_array),
        n_trials=n_trials,
        timeout=timeout,
        show_progress_bar=True
    )
    
    # Get best parameters
    best_params = study.best_params
    best_score = -study.best_value  # Convert back to positive
    
    print(f"\nOptimization completed!")
    print(f"Best composite score: {best_score:.4f}")
    print(f"Best parameters: {best_params}")
    
    # Test best parameters
    print(f"\n=== Results with Best Parameters ===")
        
    # Apply best dimensionality reduction
    if best_params['n_components'] < embeddings_array.shape[1]:
        svd = TruncatedSVD(n_components=n_components, random_state=RANDOM_STATE)
        best_embeddings = svd.fit_transform(embeddings_array)
        # Re-normalize after SVD
        best_embeddings /= np.linalg.norm(reduced_embeddings, axis=1, keepdims=True)
        print(f"Reduced dimensions from {embeddings_array.shape[1]} to {best_params['n_components']}")
    else:
        best_embeddings = embeddings_array
        reducer = None
        print("No dimensionality reduction applied")
     
    # Fit with best parameters
    best_clusterer = hdbscan.HDBSCAN(
        min_cluster_size=best_params['min_cluster_size'],
        min_samples=best_params['min_samples'],
        metric="euclidean",
        cluster_selection_method="eom",
    )
    
    best_labels = best_clusterer.fit_predict(best_embeddings)
    best_metrics = calculate_clustering_metrics(best_embeddings, best_labels, best_clusterer)
    
    print_clustering_summary(best_metrics)
    print()
    
    # Return results
    return {
        'study': study,
        'best_params': best_params,
        'best_score': best_score,
        'best_clusterer': best_clusterer,
        'best_labels': best_labels,
        'best_embeddings': best_embeddings,
        'best_metrics': best_metrics,
        'svd_transformer': svd if best_params['n_components'] < embeddings_array.shape[1] else None
    }

results = optimize_hdbscan(embeddings_df, n_trials=100)

In [ ]:
results


In [ ]:

metrics = calculate_clustering_metrics(embeddings_df.values, labels, clusterer) 
print_clustering_summary(metrics)


In [ ]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_proampt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

- cluster  articles
- combine title, description, topics, summary if present 
- fetch embeddings for summaries 
- do dimensionality reduction
- cluster with hdbscan
- show metrics

- tune dbscan
- name the clusters with topic_writer
- store the cluster names 

output is , df has topic list and topic_str, summary updated, df has cluster , state clusters updated

In [ ]:
state = await agent.run_step("get state")
state 


In [ ]:
inspect_result = await agent.run_step("inspect state")


In [ ]:
state = await agent.get_state_direct()


In [ ]:
print(status_result)
